ver 8 (6), v10

In [4]:
import tensorflow as tf

# Keras được tích hợp trong TensorFlow dưới dạng tf.keras
keras_version_from_tf = tf.keras.__version__
print(f"Phiên bản Keras API (thông qua tf.keras): {keras_version_from_tf}")

Phiên bản Keras API (thông qua tf.keras): 3.5.0


In [2]:
!cat /proc/cpuinfo | grep "model name" | uniq 
# Hoặc để xem số core
!nproc 

model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
4


In [3]:
!free -h 
# Hoặc chi tiết hơn
!cat /proc/meminfo | grep MemTotal

               total        used        free      shared  buff/cache   available
Mem:            31Gi       836Mi        23Gi       1.0Mi       6.9Gi        30Gi
Swap:             0B          0B          0B
MemTotal:       32873392 kB


In [1]:
!nvidia-smi

Thu May 22 18:58:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from typing import Tuple

import cv2
import json
from tqdm.notebook import tqdm


import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from collections import Counter
import shutil 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from typing import List, Tuple, Optional

In [ ]:
base_input_dir = "/kaggle/input/btxrd-data/BTXRD/BTXRD" 
image_dir = os.path.join(base_input_dir, "images")
annotation_dir = os.path.join(base_input_dir, "Annotations")
excel_path = "/kaggle/input/btxrd-data/classification.xlsx"


# output_dir = "/kaggle/working/btxrd-v2.2"
# output_image_dir = os.path.join(output_dir, "images")
# output_anno_dir = os.path.join(output_dir, "Annotations")

In [ ]:
# Đọc file Excel
# file_path = '/kaggle/input/btxrd-data/classification.xlsx'
df = pd.read_excel(excel_path)

# Hiển thị 10 dòng đầu tiên
df.head(10)

# **Xử lý ảnh**

In [ ]:
# in 30 ảnh trước xử lý
num_images_to_show = 30
images_per_row = 5  # Số ảnh mỗi hàng
mask_color = [255, 0, 0]  # Red

def create_mask(img_size: Tuple[int, int], ann_path: str) -> np.ndarray:
    mask = Image.new('L', img_size, 0)
    if os.path.exists(ann_path):
        try:
            with open(ann_path, 'r') as f:
                data = json.load(f)
                for shape in data.get('shapes', []):
                    points = shape.get('points', [])
                    polygon_points = [(int(x), int(y)) for x, y in points]
                    if polygon_points:
                        ImageDraw.Draw(mask).polygon(polygon_points, outline=1, fill=1)
        except Exception as e:
            print(f"Lỗi annotation {ann_path}: {e}")
    return np.array(mask)

# Lấy danh sách tất cả ảnh trong thư mục
all_filenames = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Chọn ngẫu nhiên 30 ảnh
selected_filenames = random.sample(all_filenames, min(num_images_to_show, len(all_filenames)))

# Plot ảnh với mask
plt.figure(figsize=(18, 18))  # Tăng kích thước ảnh
for i, fname in enumerate(selected_filenames):
    img_path = os.path.join(image_dir, fname)
    ann_fname = os.path.splitext(fname)[0] + '.json'
    ann_path = os.path.join(annotation_dir, ann_fname)

    try:
        img_pil = Image.open(img_path).convert('L')
        img_np = np.array(img_pil)

        mask_np = create_mask(img_pil.size, ann_path)
        color_img = np.stack([img_np] * 3, axis=-1)
        color_img[mask_np == 1] = mask_color

        # Chia bố cục thành 6 hàng và 5 cột (số ảnh mỗi hàng là 5)
        plt.subplot(6, 5, i + 1)
        plt.imshow(color_img)
        plt.axis('off')  # Tắt trục
    except Exception as e:
        print(f"Lỗi khi xử lý {fname}: {e}")
        continue

# Loại bỏ khoảng trống giữa các ảnh
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()


In [ ]:

TARGET_SIZE = 512

# base_input_dir = "/kaggle/input/btxrd-data/BTXRD/BTXRD" # Đường dẫn gốc chứa ảnh và annotation
# image_dir = os.path.join(base_input_dir, "images")      # Thư mục chứa ảnh gốc
# annotation_dir = os.path.join(base_input_dir, "Annotations") # Thư mục chứa annotation gốc

output_dir = "/kaggle/working/btxrd-v2.2"
output_image_dir = os.path.join(output_dir, "images")
output_anno_dir = os.path.join(output_dir, "annotations")

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_anno_dir, exist_ok=True)

MAX_VISUALIZATIONS = 5 # Số lượng ảnh tối đa để trực quan hóa
visualized_count = 0


def get_bounding_box(points):
    if not points:
        return None
    points_array = np.array(points)
    xmin = int(np.min(points_array[:, 0]))
    ymin = int(np.min(points_array[:, 1]))
    xmax = int(np.max(points_array[:, 0]))
    ymax = int(np.max(points_array[:, 1]))
    # Đảm bảo tọa độ không âm
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    return (xmin, ymin, xmax, ymax)

try:
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg'))]
    total_images = len(image_files)
    if total_images == 0:
        print(f"Không tìm thấy file ảnh nào trong: {image_dir}")
        exit()
    print(f"Tìm thấy {total_images} ảnh để xử lý.")
except FileNotFoundError:
    print(f"Không tìm thấy thư mục ảnh: {image_dir}")
    exit()

print(f"Bắt đầu xử lý ảnh và lưu vào: {output_dir}")
# Sử dụng tqdm để hiển thị thanh tiến trình
for file in tqdm(image_files, desc="Processing Images"):
    img_path = os.path.join(image_dir, file)
    anno_filename = file.rsplit('.', 1)[0] + '.json'
    anno_path = os.path.join(annotation_dir, anno_filename)

    # Đọc ảnh gốc
    img_orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img_orig is None:
        # print(f"Không thể đọc ảnh: {file}"
        continue
    orig_height, orig_width = img_orig.shape[:2]

    # Đọc annotation gốc 
    annotation_orig = None
    has_annotation = os.path.exists(anno_path)
    if has_annotation:
        try:
            with open(anno_path, "r", encoding="utf-8") as f:
                annotation_orig = json.load(f)
        except Exception as e:
            # print(f"Lỗi khi đọc annotation {anno_filename}: {e}")
            has_annotation = False # Coi như không có nếu đọc lỗi

    img_to_draw_orig = None
    img_to_draw_padded = None
    original_bboxes = []
    transformed_bboxes = []

    should_visualize = has_annotation and (visualized_count < MAX_VISUALIZATIONS)

    if should_visualize:
        img_to_draw_orig = cv2.cvtColor(img_orig, cv2.COLOR_GRAY2BGR) # Chuyển sang BGR để vẽ màu
        if annotation_orig and "shapes" in annotation_orig:
             for shape in annotation_orig["shapes"]:
                if shape.get("shape_type") == "rectangle" and "points" in shape and len(shape["points"]) == 2:
                     # LabelMe rectangle format uses [top-left, bottom-right]
                     p1 = shape["points"][0]
                     p2 = shape["points"][1]
                     xmin = int(min(p1[0], p2[0]))
                     ymin = int(min(p1[1], p2[1]))
                     xmax = int(max(p1[0], p2[0]))
                     ymax = int(max(p1[1], p2[1]))
                     bbox = (max(0, xmin), max(0, ymin), xmax, ymax)
                     original_bboxes.append(bbox)
                     cv2.rectangle(img_to_draw_orig, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2) # Vẽ màu đỏ (BGR)
                elif shape.get("shape_type") in ["polygon", "linestrip", "point"] and "points" in shape and shape["points"]:
                     # Lấy bounding box bao quanh các loại shape khác
                     bbox = get_bounding_box(shape["points"])
                     if bbox:
                        original_bboxes.append(bbox)
                        cv2.rectangle(img_to_draw_orig, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2) # Red

    # Resize ảnh với padding để giữ tỉ lệ
    # Tính tỉ lệ resize để cạnh dài nhất bằng TARGET_SIZE
    scale = TARGET_SIZE / max(orig_height, orig_width)
    new_width = int(orig_width * scale)
    new_height = int(orig_height * scale)

    # Đảm bảo kích thước mới không lớn hơn TARGET_SIZE
    new_width = min(new_width, TARGET_SIZE)
    new_height = min(new_height, TARGET_SIZE)

    # Resize ảnh
    img_resized = cv2.resize(img_orig, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Tính toán padding
    pad_h = TARGET_SIZE - new_height
    pad_w = TARGET_SIZE - new_width
    top = pad_h // 2
    bottom = pad_h - top
    left = pad_w // 2
    right = pad_w - left

    # Thêm padding
    # Sử dụng giá trị 0 (màu đen) cho padding vì ảnh là grayscale
    padded_img = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

    # Lưu ảnh đã xử lý
    output_img_path = os.path.join(output_image_dir, file)
    try:
        # Đảm bảo kích thước cuối cùng đúng là TARGET_SIZE x TARGET_SIZE
        if padded_img.shape[0] != TARGET_SIZE or padded_img.shape[1] != TARGET_SIZE:
             # Nếu có sai lệch nhỏ do làm tròn, resize lại lần cuối
             padded_img = cv2.resize(padded_img, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_AREA)
             # print(f"Final resize needed for {file}. Original: ({orig_width}x{orig_height}), Resized: ({new_width}x{new_height}), Padded: {padded_img.shape[:2]}")


        cv2.imwrite(output_img_path, padded_img)
    except Exception as e:
        # print(f"Lỗi khi lưu ảnh {output_img_path}: {e}") # Bỏ comment nếu cần debug
        continue # Bỏ qua ảnh này nếu không lưu được

    # Xử lý và lưu annotation
    if has_annotation and annotation_orig:
        # Tạo bản sao sâu để không ảnh hưởng annotation gốc
        annotation_new = json.loads(json.dumps(annotation_orig))

        if "shapes" in annotation_new:
            new_shapes = [] # Tạo list mới để chứa các shape đã chuyển đổi
            for shape in annotation_new["shapes"]:
                if "points" in shape and shape["points"]:
                    original_points = shape["points"]
                    new_points_transformed = []
                    valid_shape = True
                    for x, y in original_points:
                        # Áp dụng tỉ lệ resize
                        new_x = x * scale
                        new_y = y * scale
                        # Áp dụng padding offset
                        new_x += left
                        new_y += top

                        # Kiểm tra xem điểm có nằm trong ảnh mới không
                        # new_x = max(0, min(TARGET_SIZE - 1, new_x))
                        # new_y = max(0, min(TARGET_SIZE - 1, new_y))
                        new_points_transformed.append([new_x, new_y])

                    # Cập nhật điểm trong shape
                    shape["points"] = new_points_transformed
                    new_shapes.append(shape) # Thêm shape đã chuyển đổi vào list mới

                    # Tính bbox mới để trực quan hóa
                    if should_visualize:
                        new_bbox = get_bounding_box(new_points_transformed)
                        if new_bbox:
                            # Đảm bảo bbox không vượt ra ngoài TARGET_SIZE
                            xmin = max(0, min(TARGET_SIZE - 1, new_bbox[0]))
                            ymin = max(0, min(TARGET_SIZE - 1, new_bbox[1]))
                            xmax = max(0, min(TARGET_SIZE - 1, new_bbox[2]))
                            ymax = max(0, min(TARGET_SIZE - 1, new_bbox[3]))
                            # Chỉ thêm vào nếu bbox hợp lệ
                            if xmax > xmin and ymax > ymin:
                                transformed_bboxes.append((xmin, ymin, xmax, ymax))

            # Cập nhật lại danh sách shapes và kích thước ảnh trong annotation
            annotation_new["shapes"] = new_shapes
            annotation_new["imagePath"] = file # Cập nhật tên file ảnh mới
            annotation_new["imageWidth"] = TARGET_SIZE
            annotation_new["imageHeight"] = TARGET_SIZE
            
            if "imageData" in annotation_new:
                annotation_new["imageData"] = None

            # Lưu file annotation mới
            output_annotation_path = os.path.join(output_anno_dir, anno_filename)
            try:
                with open(output_annotation_path, "w", encoding="utf-8") as f:
                    json.dump(annotation_new, f, indent=4, ensure_ascii=False)
            except Exception as e:
                # print(f"Lỗi khi lưu annotation {anno_filename}: {e}") # Bỏ comment nếu cần debug
                pass # Bỏ qua nếu lưu lỗi

            if should_visualize and img_to_draw_orig is not None:
                # Chuyển ảnh đã padding sang BGR để vẽ màu
                img_to_draw_padded = cv2.cvtColor(padded_img, cv2.COLOR_GRAY2BGR)
                # Vẽ các bounding box đã biến đổi
                for bbox in transformed_bboxes:
                     # Đảm bảo tọa độ là số nguyên để vẽ
                     pt1 = (int(bbox[0]), int(bbox[1]))
                     pt2 = (int(bbox[2]), int(bbox[3]))
                     cv2.rectangle(img_to_draw_padded, pt1, pt2, (0, 255, 0), 2) # Vẽ màu xanh lá (BGR)

                # Hiển thị ảnh gốc và ảnh đã xử lý
                fig, axes = plt.subplots(1, 2, figsize=(12, 6))

                # Ảnh gốc với bbox gốc (màu đỏ)
                axes[0].imshow(cv2.cvtColor(img_to_draw_orig, cv2.COLOR_BGR2RGB)) # Chuyển BGR sang RGB cho matplotlib
                axes[0].set_title(f'Original: {file}\nSize: {orig_width}x{orig_height}')
                axes[0].axis('off')

                # Ảnh đã xử lý với bbox mới (màu xanh)
                axes[1].imshow(cv2.cvtColor(img_to_draw_padded, cv2.COLOR_BGR2RGB)) # Chuyển BGR sang RGB
                axes[1].set_title(f'Processed (Resized & Padded)\nSize: {TARGET_SIZE}x{TARGET_SIZE}')
                axes[1].axis('off')

                plt.suptitle(f"Visualization {visualized_count + 1}/{MAX_VISUALIZATIONS}")
                plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Điều chỉnh layout để tiêu đề không bị che
                plt.show()

                visualized_count += 1

print(f"Xử lý {total_images} ảnh.")
if visualized_count > 0:
    print(f"Hiển thị {visualized_count} ảnh trực quan hóa.")

In [ ]:
# hiển thị random 30 hình sau khi xử lý ảnh


# Cấu hình
image_dir_test = '/kaggle/working/btxrd-v2.2/images'
annotation_dir_test = '/kaggle/working/btxrd-v2.2/annotations'
# Cấu hình
num_images_to_show = 30
images_per_row = 5  # Số ảnh mỗi hàng
mask_color = [255, 0, 0]  # Red

def create_mask(img_size: Tuple[int, int], ann_path: str) -> np.ndarray:
    mask = Image.new('L', img_size, 0)
    if os.path.exists(ann_path):
        try:
            with open(ann_path, 'r') as f:
                data = json.load(f)
                for shape in data.get('shapes', []):
                    points = shape.get('points', [])
                    polygon_points = [(int(x), int(y)) for x, y in points]
                    if polygon_points:
                        ImageDraw.Draw(mask).polygon(polygon_points, outline=1, fill=1)
        except Exception as e:
            print(f"Lỗi annotation {ann_path}: {e}")
    return np.array(mask)

# Lấy danh sách tất cả ảnh trong thư mục
all_filenames = [f for f in os.listdir(image_dir_test) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Chọn ngẫu nhiên 30 ảnh
selected_filenames = random.sample(all_filenames, min(num_images_to_show, len(all_filenames)))

# Plot ảnh với mask
plt.figure(figsize=(18, 18))  # Tăng kích thước ảnh
for i, fname in enumerate(selected_filenames):
    img_path = os.path.join(image_dir_test, fname)
    ann_fname = os.path.splitext(fname)[0] + '.json'
    ann_path = os.path.join(annotation_dir_test, ann_fname)

    try:
        img_pil = Image.open(img_path).convert('L')
        img_np = np.array(img_pil)

        mask_np = create_mask(img_pil.size, ann_path)
        color_img = np.stack([img_np] * 3, axis=-1)
        color_img[mask_np == 1] = mask_color

        # Chia bố cục thành 6 hàng và 5 cột (số ảnh mỗi hàng là 5)
        plt.subplot(6, 5, i + 1)
        plt.imshow(color_img)
        plt.axis('off')  # Tắt trục
    except Exception as e:
        print(f"Lỗi khi xử lý {fname}: {e}")
        continue

# Loại bỏ khoảng trống giữa các ảnh
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()


# **Chia tập dữ liệu**

In [ ]:
output_split_dir = "/kaggle/working/btxrd-v2.1"

ANNOTATION_EXTENSION = ".json"

VAL_SIZE = 0.20   # 20% cho tập validation
TRAIN_SIZE = 0.70 # 70% cho tập train
TEST_SIZE = 1.0 - VAL_SIZE - TRAIN_SIZE

RANDOM_STATE = 42

In [ ]:
# Đọc Dữ liệu Phân loại từ Excel
try:
    df_classification = pd.read_excel(excel_path)
    required_columns = ['image_id', 'tumor_type', 'image_filename']
    if not all(col in df_classification.columns for col in required_columns):
        missing = [col for col in required_columns if col not in df_classification.columns]
        raise ValueError(f"File Excel thiếu các cột bắt buộc: {missing}")

    df_classification['image_id'] = df_classification['image_id'].astype(str).str.strip()
    df_classification['image_filename'] = df_classification['image_filename'].astype(str).str.strip()

    print(f"Đọc thành công {len(df_classification)} dòng")
    print(df_classification['tumor_type'].value_counts())
except FileNotFoundError:
    print(f"Không tìm thấy file Excel tại {excel_path}")
    exit()
except ValueError as ve:
    print(f"Lỗi dữ liệu trong file Excel: {ve}")
    exit()
except Exception as e:
    print(f"không xác định khi đọc file Excel: {e}")
    exit()

try:
    all_image_files = glob.glob(os.path.join(image_dir_test, "*.*"))
    annotation_files = glob.glob(os.path.join(annotation_dir_test, f"*{ANNOTATION_EXTENSION}"))

    image_basenames_actual = set(os.path.splitext(os.path.basename(f))[0] for f in all_image_files)
    annotation_basenames_actual = set(os.path.splitext(os.path.basename(f))[0] for f in annotation_files)

    print(f"Tìm thấy {len(all_image_files)} tệp")
    print(f"Tìm thấy {len(annotation_files)} tệp annotation")
except Exception as e:
    print(f"Lỗi khi quét thư mục ảnh hoặc annotation: {e}")
    exit()

In [ ]:
excel_image_ids = set(df_classification['image_id'])
valid_ids = list(excel_image_ids.intersection(image_basenames_actual).intersection(annotation_basenames_actual))

if not valid_ids:
    print("Không tìm thấy dữ liệu hợp lệ nào.")
    exit()
df_filtered = df_classification[df_classification['image_id'].isin(valid_ids)].copy()
df_filtered = df_filtered.drop_duplicates(subset=['image_id'])
filename_map = pd.Series(df_filtered.image_filename.values, index=df_filtered.image_id).to_dict()


In [ ]:
# Chuẩn bị dữ liệu (X=IDs, y=Labels) cho việc chia
X = df_filtered['image_id'].tolist() # Danh sách ID ảnh 
y = df_filtered['tumor_type'].tolist() # Danh sách nhãn tương ứng

# Chia Lần 1 (Train+Val / Test)
X_train_val, X_test, y_train_val, y_test = [], [], [], []
if len(X) < 2:
    print("Không đủ mẫu dữ liệu (< 2) để thực hiện chia.")
    exit()
if TEST_SIZE <= 0 or TEST_SIZE >= 1:
     print(f"Tỷ lệ Test ({TEST_SIZE:.2f}) không hợp lệ. Toàn bộ dữ liệu sẽ là Train+Val.")
     X_train_val, y_train_val = X, y
else:
    try:
        unique_classes_total, counts_total = np.unique(y, return_counts=True)
        stratify_option_1 = y
        if len(unique_classes_total) < 2:
            print("Chỉ có 1 lớp. Chia ngẫu nhiên cho Test.")
            stratify_option_1 = None
        elif np.any(counts_total < 2):
             print(f"Có lớp < 2 mẫu. Chia ngẫu nhiên cho Test.")
             stratify_option_1 = None

        X_train_val, X_test, y_train_val, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=stratify_option_1
        )
        print(f"Chia lần 1: {len(X_train_val)} Train+Val, {len(X_test)} Test.")
        print("Phân phối 'tumor_type' trong Test:", sorted(Counter(y_test).items()))
    except ValueError as e:
         print(f"Lỗi khi chia lần 1 (Test): {e}. Thoát.")
         exit()


# Chia lần 2 (Train / Validation)
X_train, X_val, y_train, y_val = [], [], [], []
if not X_train_val:
     print("Tập Train+Val rỗng.")
elif len(X_train_val) == 1:
     print("Tập Train+Val chỉ có 1 mẫu -> vào Train.")
     X_train, y_train = X_train_val, y_train_val
elif VAL_SIZE <= 0 or VAL_SIZE >= 1:
     print(f"Tỷ lệ Val ({VAL_SIZE:.4f}) không hợp lệ. Toàn bộ Train+Val -> Train.")
     X_train, y_train = X_train_val, y_train_val
else:
    try:
        unique_classes_tv, counts_tv = np.unique(y_train_val, return_counts=True)
        stratify_option_2 = y_train_val
        if len(unique_classes_tv) < 2:
            print("Train+Val chỉ còn 1 lớp. Chia ngẫu nhiên cho Val.")
            stratify_option_2 = None
        elif np.any(counts_tv < 2):
             print(f"Có lớp < 2 mẫu trong Train+Val. Chia ngẫu nhiên cho Val.")
             stratify_option_2 = None

        X_train, X_val, y_train, y_val = train_test_split(
            X_train_val, y_train_val, test_size=VAL_SIZE,
            random_state=RANDOM_STATE, stratify=stratify_option_2
        )
        print(f"Chia lần 2: {len(X_train)} Train, {len(X_val)} Validation.")
        print("Phân phối 'tumor_type' trong Train:", sorted(Counter(y_train).items()))
        print("Phân phối 'tumor_type' trong Validation:", sorted(Counter(y_val).items()))
    except ValueError as e:
        print(f"Lỗi khi chia lần 2 (Validation): {e}. Toàn bộ Train+Val -> Train.")
        X_train, y_train = X_train_val, y_train_val # Gán lại vào Train

In [ ]:
# kết quả sau khi chia
total_ids_split = len(X_train) + len(X_val) + len(X_test)
original_valid_count = len(df_filtered)

print(f"Tổng số mẫu hợp lệ ban đầu: {original_valid_count}")
print(f"Tổng số IDs được chia vào các tập: {total_ids_split}")
if total_ids_split != original_valid_count:
     print(f"Số ID được chia ({total_ids_split}) không khớp số ID hợp lệ ({original_valid_count}). Kiểm tra logic chia.")

print(f"Train set IDs:      {len(X_train):>5}")
print(f"Validation set IDs: {len(X_val):>5}")
print(f"Test set IDs:       {len(X_test):>5}")

if total_ids_split > 0:
    print(f"\nTỷ lệ thực tế (dựa trên IDs):")
    print(f"  Train: {len(X_train) / total_ids_split * 100:>6.1f}%")
    print(f"  Val:   {len(X_val) / total_ids_split * 100:>6.1f}%")
    print(f"  Test:  {len(X_test) / total_ids_split * 100:>6.1f}%")

print("\nPhân phối 'tumor_type' cuối cùng (dựa trên IDs đã chia):")
print(f"Train:      {sorted(Counter(y_train).items())}")
print(f"Validation: {sorted(Counter(y_val).items())}")
print(f"Test:       {sorted(Counter(y_test).items())}")

# **Huấn luyện mô hình**

In [1]:
!pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [3]:
# --- Cấu hình ---
import os # Thêm import os nếu chưa có
import numpy as np # Thêm import numpy nếu dùng trong tính mean/std
import pandas as pd # Thêm import pandas nếu dùng trong tải metadata
from tqdm import tqdm # Thêm import tqdm
import tensorflow as tf # Thêm import tensorflow
from PIL import Image, ImageDraw # Thêm import PIL
import json # Thêm import json
import matplotlib.pyplot as plt # Thêm import matplotlib nếu dùng plot_image

INPUT_DATA_ROOT = '/kaggle/input/btxrd-data' # THAY ĐỔI NẾU MÔI TRƯỜNG CỦA BẠN KHÁC
BASE_DATA_DIR = os.path.join(INPUT_DATA_ROOT, 'btxrd-v2.1')
CLASSIFICATION_FILE = os.path.join(INPUT_DATA_ROOT, 'classification.xlsx')
IMAGE_SUBDIR_NAME = 'images'
ANNOTATION_SUBDIR_NAME = 'annotations'

# Tham số Model & Huấn luyện
TARGET_SIZE = 512
N_CLASSES = 2 # 2 lớp: 0 (nền), 1 (khối u)
BATCH_SIZE = 4 # Sẽ được dùng trong config wandb
BUFFER_SIZE = 100 # Dùng cho dataset.shuffle
EPOCHS = 300 # Sẽ được dùng trong config wandb và vòng lặp for
LEARNING_RATE = 1e-4 # Sẽ được dùng trong config wandb
L2_REG_FACTOR = 1e-5
DROPOUT_RATE = 0.3

# --- Cải tiến để tăng IoU ---
USE_COMBINED_LOSS = True
DICE_LOSS_WEIGHT = 0.6
USE_FOCAL_LOSS_IN_COMBINED = True
FOCAL_LOSS_ALPHA = 0.25
FOCAL_LOSS_GAMMA = 2.0

USE_ATTENTION_UNET = False

# APPLY_POST_PROCESSING, POST_PROCESSING_KERNEL_SIZE, MIN_AREA_POST_PROCESSING
# thường dùng sau huấn luyện, không trực tiếp ảnh hưởng đến vòng lặp huấn luyện này

MODEL_CHECKPOINT_BASENAME = "unet_model"
TENSORBOARD_LOG_DIR = "./logs_unet_iou_focused"

# --- Các hằng số cho callback Keras tiêu chuẩn ---
PATIENCE_EARLY_STOPPING = 35
PATIENCE_REDUCE_LR = 12
MONITOR_METRIC_CB = 'val_dice_coef_metric_tumor' # QUAN TRỌNG: Phải khớp với key trong history.history

# --- Cấu hình WandB ---
WANDB_PROJECT_NAME = "btxrd-project" # Đặt tên project của bạn trên WandB
WANDB_ENTITY = "nganltt2333" # Đặt entity của bạn
WANDB_API_KEY = "2b7e633df37247dd52582a893eecab6314151a62"

In [4]:
def get_valid_paths(base_dir: str, split_type: str, img_filename_with_ext: str) -> Optional[Tuple[str, str]]:
    split_dir = os.path.join(base_dir, split_type); image_dir_path = os.path.join(split_dir, IMAGE_SUBDIR_NAME); annotation_dir_path = os.path.join(split_dir, ANNOTATION_SUBDIR_NAME)
    img_path = os.path.join(image_dir_path, img_filename_with_ext); base_name = os.path.splitext(img_filename_with_ext)[0]; json_filename = base_name + '.json'
    json_path = os.path.join(annotation_dir_path, json_filename)
    if os.path.exists(img_path) and os.path.exists(json_path): return img_path, json_path
    return None

def create_mask_pil(mask_size: Tuple[int, int], json_path: str) -> Image.Image:
    if not os.path.exists(json_path): return Image.new('L', (mask_size[1], mask_size[0]), 0)
    mask = Image.new('L', (mask_size[1], mask_size[0]), 0)
    try:
        with open(json_path, 'r') as f: data = json.load(f)
        if 'shapes' not in data or not isinstance(data['shapes'], list) or not data['shapes']: return mask
        for shape in data['shapes']:
             if 'points' in shape and isinstance(shape['points'], list):
                  polygon = [tuple(point) for point in shape['points']]
                  if len(polygon) >= 3: ImageDraw.Draw(mask).polygon(polygon, outline=255, fill=255)
    except (json.JSONDecodeError, Exception): return Image.new('L', (mask_size[1], mask_size[0]), 0)
    return mask

def plot_image(ax: plt.Axes, image_data: np.ndarray, title: str, cmap='gray'):
    if image_data.ndim == 2 or (image_data.ndim == 3 and image_data.shape[2] == 1): ax.imshow(image_data.squeeze(), cmap=cmap)
    else: ax.imshow(image_data)
    ax.set_title(title, fontsize=10); ax.axis('off')

all_image_paths = []; all_mask_paths = []; all_types = []
try:
    if not os.path.exists(CLASSIFICATION_FILE): raise FileNotFoundError(f"Không tìm thấy file phân loại tại {CLASSIFICATION_FILE}")
    if not os.path.isdir(BASE_DATA_DIR): raise FileNotFoundError(f"Không tìm thấy thư mục dữ liệu cơ sở: {BASE_DATA_DIR}")
    df_classification = pd.read_excel(CLASSIFICATION_FILE)
    required_cols = ['image_filename', 'type']
    if not all(col in df_classification.columns for col in required_cols): raise ValueError(f"File Excel phải chứa các cột: {required_cols}")
    for index, row in tqdm(df_classification.iterrows(), total=len(df_classification), desc="Kiểm tra file"):
        img_filename_with_ext = row['image_filename']; file_type = row['type']
        if pd.isna(img_filename_with_ext) or pd.isna(file_type) or file_type not in ['train', 'val', 'test']: continue
        paths = get_valid_paths(BASE_DATA_DIR, str(file_type).lower(), str(img_filename_with_ext))
        if paths: img_path, json_path = paths; all_image_paths.append(img_path); all_mask_paths.append(json_path); all_types.append(str(file_type).lower())
    if not all_image_paths: print("\nLỗi: Không tìm thấy cặp ảnh-chú thích hợp lệ nào."); exit()
    df_paths = pd.DataFrame({'image_path': all_image_paths, 'mask_path': all_mask_paths, 'type': all_types})
    df_train = df_paths[df_paths['type'] == 'train'].reset_index(drop=True); df_val = df_paths[df_paths['type'] == 'val'].reset_index(drop=True); df_test = df_paths[df_paths['type'] == 'test'].reset_index(drop=True)
    train_image_paths = df_train['image_path'].tolist(); train_mask_paths = df_train['mask_path'].tolist()
    val_image_paths = df_val['image_path'].tolist(); val_mask_paths = df_val['mask_path'].tolist()
    test_image_paths = df_test['image_path'].tolist(); test_mask_paths = df_test['mask_path'].tolist()
    print(f"\nPhân chia dữ liệu: Train({len(train_image_paths)}), Val({len(val_image_paths)}), Test({len(test_image_paths)})")
    if not train_image_paths: print("Cảnh báo: Tập huấn luyện rỗng!"); exit()
except Exception as e: print(f"Lỗi khi tải siêu dữ liệu: {e}"); import traceback; traceback.print_exc(); exit()

# Tính toán Mean/Std
mean_pixel = 0.5; std_pixel = 0.1
num_train_images = len(train_image_paths)
if num_train_images > 0:
    print("Đang tính toán Mean/Std...")
    pixel_sum = 0.0; pixel_sum_sq = 0.0; total_pixels_calculated = 0; processed_count = 0
    sample_size_for_stats = min(num_train_images, 250) # Tăng nhẹ sample size
    sampled_train_paths = np.random.choice(train_image_paths, size=sample_size_for_stats, replace=False)
    for img_path in tqdm(sampled_train_paths, desc="Tính Mean/Std"):
        try:
            img_bytes = tf.io.read_file(img_path); img = tf.io.decode_image(img_bytes, channels=1, expand_animations=False, dtype=tf.float32)
            img = tf.image.resize(img, [TARGET_SIZE, TARGET_SIZE])
            pixel_sum += tf.reduce_sum(img).numpy(); pixel_sum_sq += tf.reduce_sum(tf.square(img)).numpy()
            total_pixels_calculated += (TARGET_SIZE * TARGET_SIZE); processed_count += 1
        except Exception: pass
    if processed_count > 0 and total_pixels_calculated > 0:
        mean_pixel = pixel_sum / total_pixels_calculated; variance = (pixel_sum_sq / total_pixels_calculated) - (mean_pixel ** 2)
        std_pixel = np.sqrt(max(variance, 1e-7)); print(f"Mean: {mean_pixel:.4f}, Std Dev: {std_pixel:.4f}")
        if std_pixel < 1e-4: std_pixel = 0.1; print("Std Dev quá thấp, dùng mặc định 0.1.")
    else: print(f"Cảnh báo: Không tính được mean/std, dùng mặc định.")
std_pixel = max(std_pixel, 1e-7)

# Pipeline Dữ liệu TensorFlow
def load_mask_from_json_py(json_path_bytes):
    json_path = json_path_bytes.numpy().decode('utf-8'); pil_mask = create_mask_pil((TARGET_SIZE, TARGET_SIZE), json_path)
    mask_np = np.array(pil_mask, dtype=np.uint8); mask_np = (mask_np > 128).astype(np.uint8)
    return mask_np

@tf.function
def load_and_preprocess(image_path, mask_json_path):
    img_bytes = tf.io.read_file(image_path)
    try: img = tf.io.decode_image(img_bytes, channels=1, expand_animations=False, dtype=tf.float32)
    except tf.errors.InvalidArgumentError:
        try: img = tf.image.decode_png(img_bytes, channels=1, dtype=tf.uint8); img = tf.cast(img, tf.float32) / 255.0
        except tf.errors.InvalidArgumentError: img = tf.image.decode_jpeg(img_bytes, channels=1); img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, [TARGET_SIZE, TARGET_SIZE]); img.set_shape([TARGET_SIZE, TARGET_SIZE, 1])
    mask_np_binary = tf.py_function(func=load_mask_from_json_py, inp=[mask_json_path], Tout=tf.uint8)
    mask_np_binary.set_shape([TARGET_SIZE, TARGET_SIZE])
    mask_onehot = tf.one_hot(tf.cast(mask_np_binary, tf.int32), depth=N_CLASSES, dtype=tf.float32)
    mask_onehot.set_shape([TARGET_SIZE, TARGET_SIZE, N_CLASSES])
    img = (img - mean_pixel) / std_pixel
    return img, mask_onehot

@tf.function
def augment_data_tf(image, mask_onehot):
    combined = tf.concat([image, tf.cast(mask_onehot, image.dtype)], axis=-1) # Nối image và mask (đã cast về dtype của image)
    if tf.random.uniform(()) > 0.5: combined = tf.image.flip_left_right(combined)
    if tf.random.uniform(()) > 0.5: combined = tf.image.flip_up_down(combined)
    k_rot = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    combined = tf.image.rot90(combined, k=k_rot)
    img_aug = combined[..., :1]
    mask_aug = tf.cast(combined[..., 1:], tf.float32)
    img_aug = tf.image.random_brightness(img_aug, max_delta=0.25)
    img_aug = tf.image.random_contrast(img_aug, lower=0.7, upper=1.3)
    if tf.random.uniform(()) > 0.3:
        scale = tf.random.uniform((), 0.8, 1.2)
        new_height = tf.cast(TARGET_SIZE * scale, tf.int32)
        new_width = tf.cast(TARGET_SIZE * scale, tf.int32)
        img_scaled = tf.image.resize(img_aug, [new_height, new_width], method=tf.image.ResizeMethod.BILINEAR)
        mask_scaled = tf.image.resize(mask_aug, [new_height, new_width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        img_aug = tf.image.resize_with_crop_or_pad(img_scaled, TARGET_SIZE, TARGET_SIZE)
        mask_aug = tf.image.resize_with_crop_or_pad(mask_scaled, TARGET_SIZE, TARGET_SIZE)
    img_aug = tf.clip_by_value(img_aug, -3.0, 3.0)
    img_aug.set_shape([TARGET_SIZE, TARGET_SIZE, 1])
    mask_aug.set_shape([TARGET_SIZE, TARGET_SIZE, N_CLASSES])
    return img_aug, mask_aug

def create_dataset(image_paths, mask_paths, is_training=True):
    if not image_paths or not mask_paths: return tf.data.Dataset.from_tensor_slices(([], [])).batch(BATCH_SIZE)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    if is_training: dataset = dataset.shuffle(buffer_size=min(BUFFER_SIZE, len(image_paths)), reshuffle_each_iteration=True)
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if is_training: dataset = dataset.map(augment_data_tf, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=(is_training if len(image_paths) >= BATCH_SIZE else False))
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_ds = create_dataset(train_image_paths, train_mask_paths, is_training=True)
val_ds = create_dataset(val_image_paths, val_mask_paths, is_training=False)
test_ds = create_dataset(test_image_paths, test_mask_paths, is_training=False)

Kiểm tra file: 100%|██████████| 3746/3746 [00:09<00:00, 413.20it/s] 



Phân chia dữ liệu: Train(1344), Val(336), Test(187)
Đang tính toán Mean/Std...


Tính Mean/Std: 100%|██████████| 250/250 [00:03<00:00, 78.07it/s] 


Mean: 0.1915, Std Dev: 0.2355


In [5]:
# UNET
class AttentionGate(layers.Layer):
    def __init__(self, F_g, F_l, F_int, **kwargs): super(AttentionGate, self).__init__(**kwargs); self.W_g = layers.Conv2D(F_int, 1, padding='same', kernel_initializer='he_normal'); self.W_x = layers.Conv2D(F_int, 1, padding='same', kernel_initializer='he_normal'); self.psi = layers.Conv2D(1, 1, padding='same', kernel_initializer='he_normal', activation='sigmoid'); self.relu = layers.Activation('relu')
    def call(self, g, x): g1 = self.W_g(g); x1 = self.W_x(x); psi_input = self.relu(g1 + x1); alpha = self.psi(psi_input); return x * alpha
def conv_block(inputs, num_filters, l2_reg, dropout):
    x = layers.Conv2D(num_filters, 3, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs); x = layers.BatchNormalization()(x); x = layers.Activation('relu')(x)
    if dropout > 0: x = layers.Dropout(dropout)(x)
    x = layers.Conv2D(num_filters, 3, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x); x = layers.BatchNormalization()(x); x = layers.Activation('relu')(x)
    return x
def encoder_block(inputs, num_filters, l2_reg, dropout, pool=True): c = conv_block(inputs, num_filters, l2_reg, dropout); p = layers.MaxPooling2D(2)(c) if pool else None; return c, p
def decoder_block(inputs, skip_features, num_filters, l2_reg, dropout, use_attention):
    x = layers.Conv2DTranspose(num_filters, 2, strides=2, padding='same')(inputs)
    if use_attention and skip_features is not None: att_gate = AttentionGate(num_filters, skip_features.shape[-1], max(1, skip_features.shape[-1] // 2) ); skip_features = att_gate(g=x, x=skip_features)
    if skip_features is not None: x = layers.Concatenate()([x, skip_features])
    x = conv_block(x, num_filters, l2_reg, dropout); return x
def build_unet(input_shape, n_classes=N_CLASSES, l2_reg=L2_REG_FACTOR, dropout=DROPOUT_RATE, use_attention=USE_ATTENTION_UNET):
    filters = [64, 128, 256, 512, 1024]
    inputs = keras.Input(shape=input_shape); skips = []; x = inputs
    for f in filters[:-1]: s, p = encoder_block(x, f, l2_reg, dropout, pool=True); skips.append(s); x = p
    x, _ = encoder_block(x, filters[-1], l2_reg, dropout*1.3, pool=False)
    for i, f in reversed(list(enumerate(filters[:-1]))): x = decoder_block(x, skips[i], f, l2_reg, dropout, use_attention)
    outputs = layers.Conv2D(n_classes, 1, padding='same', activation='softmax')(x)
    return keras.Model(inputs, outputs, name=f"{'Attention' if use_attention else ''}UNet_filters{filters[0]}")

# --- HÀM MẤT MÁT (LOSS FUNCTIONS) ---
SMOOTH = 1e-6
def dice_coef(y_true_one_hot, y_pred_softmax):
    y_true_f = tf.keras.backend.flatten(y_true_one_hot)
    y_pred_f = tf.keras.backend.flatten(y_pred_softmax)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + SMOOTH)

def dice_coef_metric_tumor(y_true, y_pred):
    # y_true: (batch, H, W, N_CLASSES), y_pred: (batch, H, W, N_CLASSES)
    return dice_coef(y_true[..., 1], y_pred[..., 1]) if N_CLASSES >= 2 else 0.0
dice_coef_metric_tumor.__name__ = 'dice_coef_metric_tumor' # Khớp với `metrics_to_plot`

def dice_loss_tumor(y_true, y_pred):
    return 1.0 - dice_coef(y_true[..., 1], y_pred[..., 1]) if N_CLASSES >= 2 else 0.0

def iou_coef(y_true_one_hot, y_pred_softmax):
    y_true_f = tf.keras.backend.flatten(y_true_one_hot)
    y_pred_f = tf.keras.backend.flatten(y_pred_softmax)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + SMOOTH) / (union + SMOOTH)

def iou_metric_tumor(y_true, y_pred):
    return iou_coef(y_true[..., 1], y_pred[..., 1]) if N_CLASSES >= 2 else 0.0
iou_metric_tumor.__name__ = 'tumor_iou' # Khớp với `metrics_to_plot`

# --- CÁC METRICS MỚI CHO LỚP TUMOR ---
def precision_recall_tumor_base(y_true, y_pred, metric_type):
    if N_CLASSES < 2:
        return tf.constant(0.0, dtype=tf.float32)

    # Lấy kênh của lớp tumor (giả sử lớp 1 là tumor)
    y_true_tumor = y_true[..., 1] # Ground truth cho lớp tumor (0 hoặc 1)
    
    # Chuyển đổi y_pred (softmax probabilities) thành dự đoán nhãn cứng (0 hoặc 1) cho lớp tumor
    # Cách 1: Dựa trên xác suất cao nhất (argmax)
    y_pred_labels = tf.argmax(y_pred, axis=-1) # Shape: (batch, H, W)
    y_pred_tumor_binary = tf.cast(tf.equal(y_pred_labels, 1), tf.float32) # 1 nếu dự đoán là tumor (lớp 1), 0 nếu khác

    # Cách 2: (Nếu chỉ có 2 lớp, có thể dùng ngưỡng 0.5 cho xác suất lớp tumor)
    # y_pred_tumor_binary = tf.cast(y_pred[..., 1] > 0.5, tf.float32) # Chỉ phù hợp nếu N_CLASSES=2 và lớp 1 là tumor

    # Flatten để tính toán
    y_true_tumor_flat = tf.keras.backend.flatten(y_true_tumor)
    y_pred_tumor_binary_flat = tf.keras.backend.flatten(y_pred_tumor_binary)

    true_positives = tf.keras.backend.sum(y_true_tumor_flat * y_pred_tumor_binary_flat)
    
    if metric_type == 'precision':
        predicted_positives = tf.keras.backend.sum(y_pred_tumor_binary_flat)
        value = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    elif metric_type == 'recall':
        possible_positives = tf.keras.backend.sum(y_true_tumor_flat)
        value = true_positives / (possible_positives + tf.keras.backend.epsilon())
    else:
        value = tf.constant(0.0, dtype=tf.float32)
        
    return value

def precision_tumor_metric(y_true, y_pred):
    return precision_recall_tumor_base(y_true, y_pred, 'precision')
precision_tumor_metric.__name__ = 'precision_tumor' # Khớp với `metrics_to_plot`

def recall_tumor_metric(y_true, y_pred):
    return precision_recall_tumor_base(y_true, y_pred, 'recall')
recall_tumor_metric.__name__ = 'recall_tumor' # Khớp với `metrics_to_plot`
# --- KẾT THÚC METRICS MỚI ---

def categorical_focal_loss_wrapper(alpha=FOCAL_LOSS_ALPHA, gamma=FOCAL_LOSS_GAMMA):
    def focal_loss_fn(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon(); y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))
    focal_loss_fn.__name__ = f'focal_loss_alpha{alpha}_gamma{gamma}'
    return focal_loss_fn

def combined_loss_fn(y_true, y_pred, dice_w=DICE_LOSS_WEIGHT):
    d_loss = dice_loss_tumor(y_true, y_pred)
    if USE_FOCAL_LOSS_IN_COMBINED: ce_or_focal_loss = categorical_focal_loss_wrapper()(y_true, y_pred)
    else: ce_or_focal_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))
    return (dice_w * d_loss) + ((1.0 - dice_w) * ce_or_focal_loss)
combined_loss_fn.__name__ = f'combined_dice{DICE_LOSS_WEIGHT}_{"focal" if USE_FOCAL_LOSS_IN_COMBINED else "cce"}'

In [6]:
import wandb # Đảm bảo wandb đã được import
from datetime import datetime, timedelta # Để tạo tên run

# --- Build và Compile Model ---
model = build_unet((TARGET_SIZE, TARGET_SIZE, 1), N_CLASSES, L2_REG_FACTOR, DROPOUT_RATE, USE_ATTENTION_UNET)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

if USE_COMBINED_LOSS:
    loss_to_use = combined_loss_fn
else:
    if USE_FOCAL_LOSS_IN_COMBINED:
        loss_to_use = categorical_focal_loss_wrapper(alpha=FOCAL_LOSS_ALPHA, gamma=FOCAL_LOSS_GAMMA)
    else:
        loss_to_use = tf.keras.losses.CategoricalCrossentropy()
        loss_to_use.__name__ = "categorical_crossentropy" # Đặt tên nếu là object

loss_name_str = loss_to_use.__name__ if hasattr(loss_to_use, '__name__') else "custom_loss"

# --- Định nghĩa danh sách metrics cho model.compile() ---
# Đảm bảo các tên này sẽ xuất hiện trong history.history
metrics_to_compile = [ # Đổi tên biến để tránh nhầm lẫn với list dùng để log
    dice_coef_metric_tumor,
    iou_metric_tumor,
    precision_tumor_metric,
    recall_tumor_metric,
    tf.keras.metrics.MeanIoU(num_classes=N_CLASSES, name='mean_iou_all'),
    tf.keras.metrics.CategoricalAccuracy(name='acc') # Keras có thể trả về 'acc' hoặc 'categorical_accuracy'
]
# Tạo list các tên metric thực tế sẽ dùng để log (từ history.history)
# Điều này quan trọng để đảm bảo key khớp khi log thủ công
# Keras trả về tên của hàm/object metric, hoặc tên bạn đặt trong tf.keras.metrics.Metric(name='...')
# Nếu metric là một hàm, history.history sẽ dùng tên hàm.
# Nếu là một object tf.keras.metrics.Metric, nó sẽ dùng thuộc tính .name
# Đối với CategoricalAccuracy, Keras có thể dùng 'acc' hoặc 'categorical_accuracy'.
# Chúng ta sẽ xử lý điều này linh hoạt hơn trong vòng lặp log.

# Các tên metric cơ bản mà chúng ta muốn log, không bao gồm 'loss' và 'val_loss' (vì chúng luôn có)
# và 'acc'/'val_acc' (sẽ xử lý riêng)
metric_names_to_log_manually = []
for m in metrics_to_compile:
    if hasattr(m, 'name'):
        metric_names_to_log_manually.append(m.name)
    elif hasattr(m, '__name__'):
        metric_names_to_log_manually.append(m.__name__)
# Loại bỏ 'acc' nếu có, vì sẽ xử lý riêng
if 'acc' in metric_names_to_log_manually:
    metric_names_to_log_manually.remove('acc')
if 'categorical_accuracy' in metric_names_to_log_manually:
     metric_names_to_log_manually.remove('categorical_accuracy')


model.compile(optimizer=optimizer, loss=loss_to_use, metrics=metrics_to_compile)
model.summary()

# --- KHỞI TẠO WEIGHTS & BIASES ---
if WANDB_API_KEY:
    wandb.login(key=WANDB_API_KEY)
else:
    try:
        wandb.login() # Thử đăng nhập tương tác nếu không có key
    except Exception as e:
        print(f"Lỗi khi đăng nhập WandB: {e}. Vui lòng đảm bảo bạn đã đăng nhập WandB.")
        # Có thể exit() ở đây nếu WandB là bắt buộc

# Lấy giờ VN cho tên run
now_vn = datetime.utcnow() + timedelta(hours=7)
# Chỉnh sửa format tên run để không có ký tự '/' không hợp lệ cho tên file/directory
run_name_wandb = f"{MODEL_CHECKPOINT_BASENAME}_{loss_name_str}_attn{USE_ATTENTION_UNET}_" + now_vn.strftime("%d%m%Y_%H%M%S")

wandb_config = {
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "target_size": TARGET_SIZE,
    "n_classes": N_CLASSES,
    "l2_reg_factor": L2_REG_FACTOR,
    "dropout_rate": DROPOUT_RATE,
    "use_combined_loss": USE_COMBINED_LOSS,
    "dice_loss_weight": DICE_LOSS_WEIGHT,
    "use_focal_loss_in_combined": USE_FOCAL_LOSS_IN_COMBINED,
    "focal_loss_alpha": FOCAL_LOSS_ALPHA,
    "focal_loss_gamma": FOCAL_LOSS_GAMMA,
    "use_attention_unet": USE_ATTENTION_UNET,
    "architecture": model.name,
    "optimizer": type(optimizer).__name__,
    "loss_function": loss_name_str,
    "mean_pixel_train": mean_pixel, # Giả sử mean_pixel, std_pixel đã được tính
    "std_pixel_train": std_pixel,
    "monitor_metric_callbacks": MONITOR_METRIC_CB # Metric cho các Keras callback
}

wandb.init(
    project=WANDB_PROJECT_NAME,
    entity=WANDB_ENTITY,
    name=run_name_wandb,
    config=wandb_config
    # sync_tensorboard=True # Vẫn có thể dùng nếu bạn có TensorBoard callback
)

Model: "UNet_filters64"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 512, 512, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 512, 512, 64)   │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 512, 512, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 512, 512, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 512, 512, 64)   │              0 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 512, 512, 64)   │         36,928 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 512, 512, 64)   │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 512, 512, 64)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 256, 256, 64)   │              0 │ activation_1[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 256, 256, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 256, 256, 128)  │            512 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 256, 256, 128)  │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 256, 256, 128)  │              0 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 256, 256, 128)  │        147,584 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 256, 256, 128)  │            512 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 256, 256, 128)  │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 128, 128, 128)  │              0 │ activation_3[0][0]     │
│ (MaxPooling2D)       

 Total params: 31,054,210 (118.46 MB)

 Trainable params: 31,042,434 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nganltt23 (nganltt2333). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
# Callbacks Keras tiêu chuẩn (KHÔNG BAO GỒM WandbCallback)

# Đường dẫn lưu checkpoint
checkpoint_path = f"{MODEL_CHECKPOINT_BASENAME}_{run_name_wandb}.keras" # Dùng run_name_wandb để duy nhất

# MONITOR_METRIC_CB ('val_dice_coef_metric_tumor') phải là một key có trong history.history khi val_ds được dùng
keras_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, # Đổi tên tham số cho rõ ràng
        save_best_only=True,
        monitor=MONITOR_METRIC_CB,
        mode='max',
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor=MONITOR_METRIC_CB,
        patience=PATIENCE_EARLY_STOPPING,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=MONITOR_METRIC_CB,
        factor=0.3,
        patience=PATIENCE_REDUCE_LR,
        mode='max',
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=TENSORBOARD_LOG_DIR, # WandB có thể sync từ đây nếu sync_tensorboard=True trong init
        histogram_freq=1 # Có thể gây chậm, cân nhắc
    )
]

In [8]:
# Class Weights
pix_cls0 = 0; pix_cls1 = 0
# Giả sử train_mask_paths đã được tạo ở Đoạn 2
if 'train_mask_paths' in locals() and train_mask_paths: # Kiểm tra biến tồn tại
    for mask_p in tqdm(train_mask_paths, desc="Đếm pixels cho class weights"):
        try:
            m = create_mask_pil((TARGET_SIZE, TARGET_SIZE), mask_p)
            m_np = (np.array(m) > 128).astype(np.uint8)
            pix_cls0 += np.sum(m_np == 0)
            pix_cls1 += np.sum(m_np == 1)
        except Exception as e:
            print(f"Lỗi khi xử lý mask {mask_p} cho class weights: {e}")
            continue
else:
    print("Cảnh báo: train_mask_paths không tồn tại hoặc rỗng, không thể tính class weights.")

class_weights = None # Khởi tạo class_weights
if pix_cls1 > 0 and pix_cls0 > 0:
    total_pix = float(pix_cls0 + pix_cls1)
    w0 = (total_pix / (N_CLASSES * float(pix_cls0)))
    w1 = (total_pix / (N_CLASSES * float(pix_cls1)))
    class_weights = {0: w0, 1: w1} # Gán giá trị cho class_weights
    print(f"Class weights đã tính: Lớp 0: {w0:.4f}, Lớp 1: {w1:.4f}")
    if w1 < w0 :
        print("Cảnh báo: Trọng số lớp khối u (1) nhỏ hơn lớp nền (0). Kiểm tra lại số lượng pixel hoặc dữ liệu.")
    if wandb.run:
        wandb.config.update({"class_weight_0": w0, "class_weight_1": w1, "calculated_class_weights": True})
else:
    print("Không tính được class weights (số pixel lớp 0 hoặc 1 bằng 0 hoặc train_mask_paths rỗng). Sử dụng None.")
    if wandb.run:
        wandb.config.update({"calculated_class_weights": False})

Đếm pixels cho class weights: 100%|██████████| 1344/1344 [00:05<00:00, 250.28it/s]

Class weights đã tính: Lớp 0: 0.5089, Lớp 1: 28.6592


In [ ]:
# Huấn luyện Model với vòng lặp thủ công và log thủ công lên WandB

# Kiểm tra sự tồn tại của train_ds và val_ds (nếu val_image_paths có)
if 'train_ds' not in locals() or not train_ds:
    print("Lỗi: Tập huấn luyện (train_ds) chưa được tạo hoặc rỗng.")
    if wandb.run: wandb.finish(exit_code=1)
    exit()

use_validation = 'val_image_paths' in locals() and val_image_paths and 'val_ds' in locals() and val_ds
if 'val_image_paths' in locals() and val_image_paths and ('val_ds' not in locals() or not val_ds):
    print("Lỗi: Có val_image_paths nhưng tập validation (val_ds) chưa được tạo hoặc rỗng.")
    if wandb.run: wandb.finish(exit_code=1)
    exit()

print(f"\nBắt đầu huấn luyện cho {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch + 1}/{EPOCHS} ---")

    history = model.fit(
        train_ds,
        validation_data=val_ds if use_validation else None,
        epochs=1, # CHỈ HUẤN LUYỆN 1 EPOCH MỖI LẦN GỌI FIT
        class_weight=class_weights, # Từ Đoạn 6
        callbacks=keras_callbacks, # Callbacks Keras tiêu chuẩn từ Đoạn 5
        verbose=1
    )

    current_logs = history.history
    if not current_logs:
        print(f"Cảnh báo: Không có logs nào được trả về từ model.fit() ở epoch {epoch + 1}.")
        continue

    # --- Ghi log thủ công cho W&B ---
    log_data_to_wandb = {"epoch": epoch + 1}

    # Metrics huấn luyện
    log_data_to_wandb["loss"] = current_logs.get("loss", [None])[0]
    # Xử lý 'acc' hoặc 'categorical_accuracy' cho training
    train_acc_key = None
    if "acc" in current_logs:
        train_acc_key = "acc"
    elif "categorical_accuracy" in current_logs:
        train_acc_key = "categorical_accuracy"
    if train_acc_key:
        log_data_to_wandb[train_acc_key] = current_logs.get(train_acc_key, [None])[0]

    # Log các metrics tùy chỉnh khác cho training
    for metric_name in metric_names_to_log_manually: # Từ Đoạn 4
        if metric_name in current_logs:
            log_data_to_wandb[metric_name] = current_logs.get(metric_name, [None])[0]


    # Metrics validation (nếu có)
    if use_validation:
        log_data_to_wandb["val_loss"] = current_logs.get("val_loss", [None])[0]
        # Xử lý 'val_acc' hoặc 'val_categorical_accuracy'
        val_acc_key = None
        if "val_acc" in current_logs:
            val_acc_key = "val_acc"
        elif "val_categorical_accuracy" in current_logs:
            val_acc_key = "val_categorical_accuracy"
        if val_acc_key:
            log_data_to_wandb[val_acc_key] = current_logs.get(val_acc_key, [None])[0]

        # Log các metrics tùy chỉnh khác cho validation
        for metric_name in metric_names_to_log_manually: # Từ Đoạn 4
            val_metric_key = f"val_{metric_name}"
            if val_metric_key in current_logs:
                log_data_to_wandb[val_metric_key] = current_logs.get(val_metric_key, [None])[0]

    wandb.log(log_data_to_wandb)
    print(f"Đã log metrics cho epoch {epoch + 1} lên WandB.")

    # Kiểm tra điều kiện dừng sớm từ EarlyStopping callback
    if model.stop_training:
        print(f"Huấn luyện dừng sớm bởi EarlyStopping callback sau epoch {epoch + 1}.")
        break

print("\nHuấn luyện hoàn tất (hoặc dừng sớm)!")

# Kết thúc run WandB
if wandb.run:
    # (Tùy chọn) Log model tốt nhất như một artifact
    # Giả sử ModelCheckpoint đã lưu model tốt nhất vào checkpoint_path
    if os.path.exists(checkpoint_path):
        print(f"Đang log model tốt nhất từ: {checkpoint_path}")
        best_model_artifact = wandb.Artifact(
            f'{MODEL_CHECKPOINT_BASENAME}-best_model',
            type='model',
            description=f'Best model based on {MONITOR_METRIC_CB} from run {run_name_wandb}',
            metadata=dict(wandb.config) # Lưu config của run vào metadata artifact
        )
        best_model_artifact.add_file(checkpoint_path)
        wandb.log_artifact(best_model_artifact)
        print("Đã log model tốt nhất lên WandB Artifacts.")
    else:
        print(f"Không tìm thấy model checkpoint tại: {checkpoint_path} để log artifact.")

    wandb.finish()


Bắt đầu huấn luyện cho 300 epochs...

--- Epoch 1/300 ---
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step - acc: 0.8005 - dice_coef_metric_tumor: 0.0652 - loss: 0.6923 - mean_iou_all: 0.2516 - precision_tumor: 0.0583 - recall_tumor: 0.6086 - tumor_iou: 0.0344
Epoch 1: val_dice_coef_metric_tumor improved from -inf to 0.08014, saving model to unet_model_unet_model_combined_dice0.6_focal_attnFalse_31052025_221757.keras
336/336 ━━━━━━━━━━━━━━━━━━━━ 219s 483ms/step - acc: 0.8006 - dice_coef_metric_tumor: 0.0653 - loss: 0.6922 - mean_iou_all: 0.2516 - precision_tumor: 0.0583 - recall_tumor: 0.6085 - tumor_iou: 0.0344 - val_acc: 0.9001 - val_dice_coef_metric_tumor: 0.0801 - val_loss: 0.6192 - val_mean_iou_all: 0.2487 - val_precision_tumor: 0.0561 - val_recall_tumor: 0.3082 - val_tumor_iou: 0.0431 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 1.
Đã log metrics cho epoch 1 lên WandB.

--- Epoch 2/300 ---
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - acc: 0.8839 